In [1]:
import pandas as pd
import numpy as np
import os 
import string 

In [42]:
###User defined parameters###
# path = 'C:\\Users\\noxtu\\Data Science Projects\\Data-brenntag\\2022\\' #path name of folder containing all the specific year drum filling report only
path = 'C:\\Users\\User\\Documents\\GitHub\\Brenntag\\'

######
file_name = os.listdir(path)

In [43]:
#Ignore warnings from openpyxl
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [44]:
year_df = pd.DataFrame()
for file in range(len(file_name)):
    # Read excel to dataframe dictionary
    df = pd.read_excel(path+file_name[file],engine="openpyxl",sheet_name=None)

    #Obtain a list of excel sheet name
    sname = list(df.keys())
    print("File {}: Sheet names: {}".format(file+1, sname))
    new_df = pd.DataFrame()
    #Open the selected sheet name
    for i in range(len(sname)):
        cdf=df.get(sname[i])
        col_names = list(cdf.iloc[1].astype(str))
        col_names = [x.strip() for x in col_names] #Strip leading and trailing whitespaces in the column names
        #Replace header from row 1
        cdf.columns = col_names
        #Drop row 0 and 1
        cdf = cdf.drop([0,1])
        cdf = cdf[cdf['Finished Quantity'].notna()].reset_index(drop=True)
        #Change the UOM away to another name since there is duplicate
        # c = cdf.columns.values
        col_names[15]='Unit'
        cdf.columns = col_names
        c = ['Date Of Filling','Product Name','Original Quantity','Finished Quantity','UOM','Packaging','Flush','Gain','Loss','Partial','Total Manpower']
        #Filter required columns
        cdf = cdf[c]
        for a in c:
            if cdf[a].isnull().any():
                temp = cdf[cdf[a].isnull()]
                index_val = cdf[cdf[a].isnull()].index
                for k in index_val:
                    cdf[a].iloc[k] = cdf[a].iloc[k-1]
        
        #Change date format
        cdf['Date Of Filling'] = pd.to_datetime(cdf['Date Of Filling'],dayfirst=True)
        cdf['Date Of Filling'] = cdf['Date Of Filling'].dt.strftime('%d/%m/%Y')
        cdf['Month'] = file_name[file][:2]
        new_df = pd.concat([new_df,cdf],ignore_index=True) 

    year_df = pd.concat([year_df,new_df],ignore_index=True) 


File 1: Sheet names: ['1Jan to 30 Jan 22']
File 2: Sheet names: ['1-6', '14 - 19', '21 - 26', '28']
File 3: Sheet names: ['1-6', '7-13', '14-20', '21-27', '28-31']
File 4: Sheet names: ['1-30']
File 5: Sheet names: ['01 - 31 MAY 2022']
File 6: Sheet names: ['1to 30 june']
File 7: Sheet names: ['1 - 31 JULY 2022']
File 8: Sheet names: ['1-7', '8-14', '15-21', '22-28', '29-31']
File 9: Sheet names: ['1-4', '5-11', '12-18', '19-25', '26-30']
File 10: Sheet names: ['1-2', '3-9', '10-16', '17-23', '24-31']
File 11: Sheet names: ['1-6', '7-13', '14-20', '21-27', '28-30']
File 12: Sheet names: ['1-4', '5-11']


In [45]:
calculate_col = ["Finished Quantity","Packaging","Flush","Gain","Loss","Partial"]
year_df[calculate_col] = year_df[calculate_col].replace("-",0)
temp_df = year_df[~year_df["UOM"].isin(["BULK","BLC"])]
temp_df[calculate_col] = temp_df[calculate_col].astype(float)
temp_df["Tonnes"] = 0.001*((temp_df["Finished Quantity"]*temp_df["Packaging"])-temp_df["Flush"]+temp_df["Gain"]-temp_df["Loss"]+temp_df["Partial"])

temp_df2 = year_df[year_df["UOM"].isin(["BULK","BLC"])]
temp_df2["Tonnes"] = temp_df2["Finished Quantity"]*0.001

filtered_df = pd.DataFrame()
filtered_df = pd.concat([temp_df,temp_df2])
filtered_df['Date Of Filling'] = pd.to_datetime(filtered_df['Date Of Filling'],dayfirst=True)
filtered_df 

C:\Users\User\AppData\Local\Temp\ipykernel_16692\688960798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[calculate_col] = temp_df[calculate_col].astype(float)
C:\Users\User\AppData\Local\Temp\ipykernel_16692\688960798.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["Tonnes"] = 0.001*((temp_df["Finished Quantity"]*temp_df["Packaging"])-temp_df["Flush"]+temp_df["Gain"]-temp_df["Loss"]+temp_df["Partial"])
C:\Users\User\AppData\Local\Temp\ipykernel_16692\688960798.py:8: SettingWithCopy

,Date Of Filling,Product Name,Original Quantity,Finished Quantity,UOM,Packaging,Flush,Gain,Loss,Partial,Total Manpower,Month,Tonnes
1,2022-01-03,ACTREL 1140L,31.6,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040
2,2022-01-04,ISOPAR H,31.2,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040
3,2022-01-04,EXXSOL DSP 80/100,105.45,8.0,TIN,18.0,0.0,0.0,0.0,0.0,2,01,0.144
5,2022-01-04,CENTERING OIL,209,1.0,DRUM,209.0,0.0,0.0,0.0,0.0,2,01,0.209
6,2022-01-05,PRIMOL 352,19740,109.0,DRUM,180.0,180.0,0.0,0.0,0.0,3,01,19.440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2021-11-16,PRIMER HX100,15716,15716.0,BLC,15716.0,0.0,0.0,0.0,0.0,2,11,15.716
777,2021-11-16,ISOMER XYLENE,17940,17940.0,BLC,17940.0,0.0,0.0,0.0,0.0,2,11,17.940
782,2021-11-18,ISOMER XYLENE,19000,19000.0,BLC,19000.0,0.0,0.0,0.0,0.0,2,11,19.000
789,2021-11-23,PRIMER HX100,15893,15893.0,BLC,15893.0,0.0,0.0,0.0,0.0,2,11,15.893


In [46]:
#Identify similar product names
pName = sorted(list(filtered_df['Product Name'].astype(str).unique()))
print(len(pName),pName)
#Identify weight difference between Original quantity vs finished quantity
#Create new df with >5% difference
#Remove difference from overall df


137 ['15W40', 'ACETONE', 'ACETONE ', 'ACTREL 1100 NHD', 'ACTREL 1100NHD', 'ACTREL 1140L', 'ACTREL 3356L', 'ACTREL ED 33', 'ACTREL ED 33 ', 'ACTREL ED33', 'ACTREL ED33 FLUID', 'ACTRELL 1100NHD', 'ACTRELL 3356L', 'APF310D', 'APF330D', 'BCPM BLEND', 'BLENDING TANK', 'BioMGO', 'CENTERING OIL', 'CERTEREX 39', 'CERTREX 39', 'D2000', 'D95', 'DELVAC 15W40', 'DSP 80/100', 'DSP 80/100 FLUID', 'DSP 80/100 → SOLVENT 1425', 'EPF S30', 'EPF S95', 'EPFS95', 'ESCAID 110', 'ESCAID 110 FLUID', 'ETHYL ACETATE', 'EXXSOL 80 FLUID', 'EXXSOL D 40 FLUID', 'EXXSOL D110', 'EXXSOL D110 FLUID', 'EXXSOL D130', 'EXXSOL D40 FLUID', 'EXXSOL D60 FLUID', 'EXXSOL D80', 'EXXSOL D80 FLUID', 'EXXSOL D95', 'EXXSOL D95 FLUID', 'EXXSOL DSP 80/100', 'EXXSOL DSP80/100', 'EXXSOL HEPTANE', 'EXXSOL HEPTANE FLUID', 'EXXSOL HEXANE', 'EXXSOL HEXANE FLUID', 'GRADE 40%', 'HEPTANE', 'IC6', 'IH5C', 'IPA', 'IPA  ( LG )', 'IPA ( EM )', 'IPA ( EM ) ', 'IPA ( LG )', 'IPA ( O )', 'IPA (LG)', 'IPA 70%', 'IPA LG ', 'ISO HEXANE', 'ISOMER XYLENE'

In [47]:
pName_map = dict(zip(pName,pName)) #Create a mapping: actual product name map to new product name (to tidy up different product names)
### Removal of whitespace
print("Initial list size: {}".format(len(pName)))
filter1 = [x.strip() for x in pName]
 
### Remove FLUID keyword from product name
filter2 = []
for i in range(len(filter1)):
    if "FLUID" in filter1[i]:
        filter1[i] = filter1[i][:-6]
    filter2.append(filter1[i])

### Check for similar names
key_status = dict.fromkeys(pName,0)
keys = list(key_status.keys())
for k in range(len(keys)-1):
    remove = string.whitespace
    mapping = {ord(c): None for c in remove}
    if key_status[keys[k]] == 0:
        for j in range(k+1,len(keys)):
            if keys[k].translate(mapping) == keys[j].translate(mapping):
                pName_map[keys[j]] = keys[k]
                key_status[keys[j]] = 1
        key_status[keys[k]] = 1

pName_map['ACTRELL 1100NHD'] = 'ACTREL 1100 NHD'
pName_map['ACTRELL 3356L'] = 'ACTREL 3356L'
pName_map['CERTREX 39']= 'CERTEREX 39'
pName_map['SOLVESS0 150 ND'] = 'SOLVESSO 150 ND'
print(pName_map) 
#if front 3 and back 3
#remove fluid at the back

Initial list size: 137
{'15W40': '15W40', 'ACETONE': 'ACETONE', 'ACETONE ': 'ACETONE', 'ACTREL 1100 NHD': 'ACTREL 1100 NHD', 'ACTREL 1100NHD': 'ACTREL 1100 NHD', 'ACTREL 1140L': 'ACTREL 1140L', 'ACTREL 3356L': 'ACTREL 3356L', 'ACTREL ED 33': 'ACTREL ED 33', 'ACTREL ED 33 ': 'ACTREL ED 33', 'ACTREL ED33': 'ACTREL ED 33', 'ACTREL ED33 FLUID': 'ACTREL ED33 FLUID', 'ACTRELL 1100NHD': 'ACTREL 1100 NHD', 'ACTRELL 3356L': 'ACTREL 3356L', 'APF310D': 'APF310D', 'APF330D': 'APF330D', 'BCPM BLEND': 'BCPM BLEND', 'BLENDING TANK': 'BLENDING TANK', 'BioMGO': 'BioMGO', 'CENTERING OIL': 'CENTERING OIL', 'CERTEREX 39': 'CERTEREX 39', 'CERTREX 39': 'CERTEREX 39', 'D2000': 'D2000', 'D95': 'D95', 'DELVAC 15W40': 'DELVAC 15W40', 'DSP 80/100': 'DSP 80/100', 'DSP 80/100 FLUID': 'DSP 80/100 FLUID', 'DSP 80/100 → SOLVENT 1425': 'DSP 80/100 → SOLVENT 1425', 'EPF S30': 'EPF S30', 'EPF S95': 'EPF S95', 'EPFS95': 'EPF S95', 'ESCAID 110': 'ESCAID 110', 'ESCAID 110 FLUID': 'ESCAID 110 FLUID', 'ETHYL ACETATE': 'ETHYL

In [48]:
new_names = ['ACTREL','APF','DSP','EPF','ESCAID','EXXSOL','IPA','ISOPAR','MARCOL','MX','PRIMOL','PRINT-FL82','PUNCHING OIL','SOLVENT','SOLVESSO','SPF']
pName_map_simplified = dict(zip(pName_map.values(),pName_map.values()))
key_names = list(pName_map_simplified.keys())
for k in new_names:
    for i in range(len(key_names)):
        if k in key_names[i]:
            pName_map_simplified[key_names[i]] = k
        
pName_map_simplified['ISOPROPYL ALCOHOL'] = 'IPA'
pName_map_simplified['ISOPROPYL ALCOHOL ( LG )'] = 'IPA'
pName_map_simplified['ISOPROPYL ALCOHOL (EM)'] = 'IPA'

print(pName_map_simplified)

{'15W40': '15W40', 'ACETONE': 'ACETONE', 'ACTREL 1100 NHD': 'ACTREL', 'ACTREL 1140L': 'ACTREL', 'ACTREL 3356L': 'ACTREL', 'ACTREL ED 33': 'ACTREL', 'ACTREL ED33 FLUID': 'ACTREL', 'APF310D': 'APF', 'APF330D': 'APF', 'BCPM BLEND': 'BCPM BLEND', 'BLENDING TANK': 'BLENDING TANK', 'BioMGO': 'BioMGO', 'CENTERING OIL': 'CENTERING OIL', 'CERTEREX 39': 'CERTEREX 39', 'D2000': 'D2000', 'D95': 'D95', 'DELVAC 15W40': 'DELVAC 15W40', 'DSP 80/100': 'DSP', 'DSP 80/100 FLUID': 'DSP', 'DSP 80/100 → SOLVENT 1425': 'SOLVENT', 'EPF S30': 'EPF', 'EPF S95': 'EPF', 'ESCAID 110': 'ESCAID', 'ESCAID 110 FLUID': 'ESCAID', 'ETHYL ACETATE': 'ETHYL ACETATE', 'EXXSOL 80 FLUID': 'EXXSOL', 'EXXSOL D 40 FLUID': 'EXXSOL', 'EXXSOL D110': 'EXXSOL', 'EXXSOL D110 FLUID': 'EXXSOL', 'EXXSOL D130': 'EXXSOL', 'EXXSOL D60 FLUID': 'EXXSOL', 'EXXSOL D80': 'EXXSOL', 'EXXSOL D80 FLUID': 'EXXSOL', 'EXXSOL D95': 'EXXSOL', 'EXXSOL D95 FLUID': 'EXXSOL', 'EXXSOL DSP 80/100': 'EXXSOL', 'EXXSOL HEPTANE': 'EXXSOL', 'EXXSOL HEPTANE FLUID': '

In [49]:
filtered_df['Filtered Product Name'] = filtered_df['Product Name'].map(pName_map)
filtered_df['Simplified Product Name'] = filtered_df['Filtered Product Name'].map(pName_map_simplified)
filtered_df 

,Date Of Filling,Product Name,Original Quantity,Finished Quantity,UOM,Packaging,Flush,Gain,Loss,Partial,Total Manpower,Month,Tonnes,Filtered Product Name,Simplified Product Name
1,2022-01-03,ACTREL 1140L,31.6,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040,ACTREL 1140L,ACTREL
2,2022-01-04,ISOPAR H,31.2,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040,ISOPAR H,ISOPAR
3,2022-01-04,EXXSOL DSP 80/100,105.45,8.0,TIN,18.0,0.0,0.0,0.0,0.0,2,01,0.144,EXXSOL DSP 80/100,EXXSOL
5,2022-01-04,CENTERING OIL,209,1.0,DRUM,209.0,0.0,0.0,0.0,0.0,2,01,0.209,CENTERING OIL,CENTERING OIL
6,2022-01-05,PRIMOL 352,19740,109.0,DRUM,180.0,180.0,0.0,0.0,0.0,3,01,19.440,PRIMOL 352,PRIMOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2021-11-16,PRIMER HX100,15716,15716.0,BLC,15716.0,0.0,0.0,0.0,0.0,2,11,15.716,PRIMER HX 100,PRIMER HX 100
777,2021-11-16,ISOMER XYLENE,17940,17940.0,BLC,17940.0,0.0,0.0,0.0,0.0,2,11,17.940,ISOMER XYLENE,ISOMER XYLENE
782,2021-11-18,ISOMER XYLENE,19000,19000.0,BLC,19000.0,0.0,0.0,0.0,0.0,2,11,19.000,ISOMER XYLENE,ISOMER XYLENE
789,2021-11-23,PRIMER HX100,15893,15893.0,BLC,15893.0,0.0,0.0,0.0,0.0,2,11,15.893,PRIMER HX 100,PRIMER HX 100


In [51]:
filtered_df['Original Quantity'] = filtered_df['Original Quantity'].astype(float)
filtered_df['Percentage Difference'] = ((filtered_df['Tonnes'] - (filtered_df['Original Quantity']*0.001))/(filtered_df['Original Quantity']*0.001))*100
filtered_df 

,Date Of Filling,Product Name,Original Quantity,Finished Quantity,UOM,Packaging,Flush,Gain,Loss,Partial,Total Manpower,Month,Tonnes,Filtered Product Name,Simplified Product Name,Percentage Difference
1,2022-01-03,ACTREL 1140L,31.60,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040,ACTREL 1140L,ACTREL,26.582278
2,2022-01-04,ISOPAR H,31.20,2.0,PAIL,20.0,0.0,0.0,0.0,0.0,2,01,0.040,ISOPAR H,ISOPAR,28.205128
3,2022-01-04,EXXSOL DSP 80/100,105.45,8.0,TIN,18.0,0.0,0.0,0.0,0.0,2,01,0.144,EXXSOL DSP 80/100,EXXSOL,36.557610
5,2022-01-04,CENTERING OIL,209.00,1.0,DRUM,209.0,0.0,0.0,0.0,0.0,2,01,0.209,CENTERING OIL,CENTERING OIL,0.000000
6,2022-01-05,PRIMOL 352,19740.00,109.0,DRUM,180.0,180.0,0.0,0.0,0.0,3,01,19.440,PRIMOL 352,PRIMOL,-1.519757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2021-11-16,PRIMER HX100,15716.00,15716.0,BLC,15716.0,0.0,0.0,0.0,0.0,2,11,15.716,PRIMER HX 100,PRIMER HX 100,0.000000
777,2021-11-16,ISOMER XYLENE,17940.00,17940.0,BLC,17940.0,0.0,0.0,0.0,0.0,2,11,17.940,ISOMER XYLENE,ISOMER XYLENE,0.000000
782,2021-11-18,ISOMER XYLENE,19000.00,19000.0,BLC,19000.0,0.0,0.0,0.0,0.0,2,11,19.000,ISOMER XYLENE,ISOMER XYLENE,0.000000
789,2021-11-23,PRIMER HX100,15893.00,15893.0,BLC,15893.0,0.0,0.0,0.0,0.0,2,11,15.893,PRIMER HX 100,PRIMER HX 100,0.000000


In [53]:
filtered_df.to_excel(path+"2022 Drum Fillings.xlsx",index=False)

In [56]:
new_df = filtered_df[(filtered_df['Percentage Difference']<=5) & (filtered_df['Percentage Difference']>=-5)]
with pd.ExcelWriter(path+"2022 Drum Fillings.xlsx") as writer:
    filtered_df.to_excel(writer, sheet_name='All Data',index=False)
    new_df.to_excel(writer, sheet_name='5% Difference',index=False)